<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_human_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

# 자동 생성된 상담원 채팅: 코드 생성, 실행, 디버깅 및 인적 피드백을 통한 작업 해결

자동 생성은 자동화된 채팅을 통해 공동으로 작업을 수행하는 데 사용할 수 있는 LLM, 툴 또는 사람으로 구동되는 대화형 에이전트를 제공합니다. 이 프레임워크는 멀티 에이전트 대화를 통해 툴 사용과 사람 참여를 허용합니다.
이 기능에 대한 설명서는 [여기](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)를 참조하세요.

이 노트북에서는 `AssistantAgent`와 `UserProxyAgent`를 사용하여 사람의 피드백을 받아 어려운 수학 문제를 해결하는 방법을 보여드리겠습니다. 여기서 `AssistantAgent`는 사용자가 주어진 작업을 실행할 수 있도록 Python 코드를 작성할 수 있는 LLM 기반 에이전트입니다(Python 코딩 블록에서). UserProxyAgent`는 사용자가 `AssistantAgent`가 작성한 코드를 실행할 수 있도록 프록시 역할을 하는 에이전트입니다. 휴먼_입력_모드`를 적절히 설정하면 `사용자 프록시 에이전트`는 사용자에게 `AssistantAgent`에 피드백을 요청할 수도 있습니다. 예를 들어, `human_input_mode`를 "항상"으로 설정하면 `UserProxyAgent`는 항상 사용자에게 피드백을 요청합니다. 사용자 피드백이 제공되면 `사용자 프록시 에이전트`는 피드백을 `어시스턴트 에이전트`에 직접 전달합니다. 사용자 피드백이 제공되지 않으면 `UserProxyAgent`는 `AssistantAgent`가 작성한 코드를 실행하고 실행 결과(성공 또는 실패 및 해당 출력)를 `AssistantAgent`에 반환합니다.

## 요구 사항

자동 생성은 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 설치하세요:
```bash
pip 설치 pyautogen
```

In [1]:
# %pip install pyautogen~=0.1.0

## API 엔드포인트 설정

`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 함수는 환경 변수 또는 json 파일에서 구성 목록을 로드합니다.

In [2]:
import autogen

config_list = autogen.config_list_from_json("OAI_CONFIG_LIST")

먼저 유효한 json 문자열이어야 하는 환경 변수 "OAI_CONFIG_LIST"를 찾습니다. 해당 변수를 찾을 수 없으면 "OAI_CONFIG_LIST"라는 이름의 json 파일을 찾습니다.

구성 목록은 다음과 같습니다:
```python
config_list = [
    {
        '모델': 'gpt-4',
        'api_key': '<여기에 OpenAI API 키 입력>',
    }, # gpt-4용 OpenAI API 엔드포인트
    {
        'model': 'gpt-4',
        'api_key': '<여기에 첫 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 첫 번째 Azure OpenAI API 베이스>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-4용 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-4',
        'api_key': '<여기에 두 번째 Azure OpenAI API 키>',
        'api_base': '<두 번째 Azure OpenAI API 베이스는 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-4에 대한 또 다른 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 OpenAI API 키>',
    }, # gpt-3.5-turbo용 OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 첫 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 첫 번째 Azure OpenAI API 기반>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-3.5-turbo용 Azure OpenAI API 엔드포인트
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<여기에 두 번째 Azure OpenAI API 키>',
        'api_base': '<두 번째 Azure OpenAI API 기반은 여기>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # gpt-3.5-turbo에 대한 또 다른 Azure OpenAI API 엔드포인트
]
```

콜랩에서 이 노트북을 열면 왼쪽 패널의 파일 아이콘을 클릭한 다음 "파일 업로드" 아이콘을 선택해 파일을 업로드할 수 있습니다.

YAML 파일에서 불러오기 등 원하는 다른 방법으로 config_list의 값을 설정할 수 있습니다.

## 에이전트 생성

어시스턴트 에이전트와 사용자 프록시 에이전트를 구성합니다.

In [3]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 41,
        "config_list": config_list,
    }
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="ALWAYS",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
)

# the purpose of the following line is to log the conversation history
autogen.ChatCompletion.start_logging()


## 작업 수행

사용자 프록시 에이전트의 `initiate_chat()` 메서드를 호출하여 대화를 시작합니다. 아래 셀을 실행하면 어시스턴트 에이전트로부터 메시지를 받은 후 피드백을 제공하라는 메시지가 표시됩니다. 피드백을 제공하지 않으면(직접 Enter 키를 누르는 등) 사용자 프록시 에이전트는 사용자 프록시 에이전트가 제안한 코드를 대신 실행하려고 시도하거나, 메시지 끝에 "종료" 신호를 보내면 종료합니다.

In [4]:
math_problem_to_solve = """
Find $a + b + c$, given that $x+y \\neq -1$ and 
\\begin{align}
	ax + by + c & = x + 7,\\
	a + bx + cy & = 2x + 6y,\\
	ay + b + cx & = 4x + y.
\\end{align}.
"""

# the assistant receives a message from the user, which contains the task description
user_proxy.initiate_chat(assistant, message=math_problem_to_solve)

user_proxy (to assistant):


Find $a + b + c$, given that $x+y \neq -1$ and 
\begin{align}
	ax + by + c & = x + 7,\
	a + bx + cy & = 2x + 6y,\
	ay + b + cx & = 4x + y.
\end{align}.


--------------------------------------------------------------------------------
assistant (to user_proxy):

To solve the system of linear equations, I will use NumPy library in Python to calculate the coefficients and constants and then use the linalg.solve() function to find the values of a, b, and c. Finally, I will add them up to find $a + b + c$.

Step 1: Define the coefficients and constants

Step 2: Solve the system of equations using NumPy

Step 3: Calculate the sum of a, b, and c

--------------------------------------------------------------------------------
user_proxy (to assistant):

use sympy

--------------------------------------------------------------------------------
assistant (to user_proxy):

Sure, I will use Sympy to solve the system of linear equations.

Step 1: Import Sympy library

## 대화 분석하기

실제 사용자는 각 단계에서 피드백을 제공할 수 있습니다. 실제 사용자가 피드백을 제공하지 않으면 코드가 실행됩니다. 실행된 결과와 오류 메시지는 어시스턴트에게 반환되며, 어시스턴트는 피드백을 바탕으로 코드를 수정할 수 있습니다. 결국 작업이 완료되고 어시스턴트에서 "종료" 신호가 전송됩니다. 사용자가 피드백을 건너뛰면 대화가 종료됩니다.

대화가 완료되면 두 상담원 간의 대화 로그를 저장할 수 있습니다. 로그는 `autogen.ChatCompletion.logged_history`에서 확인할 수 있습니다.

In [5]:
print(autogen.ChatCompletion.logged_history)

{'[{"content": "You are a helpful AI assistant.\\nSolve tasks using your coding and language skills.\\nIn the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.\\n    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.\\n    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.\\nSolve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.\\nWhen using code, you must indicate the script type in the code block. The user cannot provide any other feed

In [6]:
import json

json.dump(autogen.ChatCompletion.logged_history, open("conversations.json", "w"), indent=2)